In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import seaborn as sns

# ========== CONFIGURACIÓN INICIAL ==========
# Tasa de cambio actual
USD_TO_PYG = 6800  # 1 USD = 6,800 PYG (actualizado)
SALARIO_MINIMO = 2_818_316  # Salario mínimo mensual en Paraguay (2024)
MEDIANA_SALARIO_OBJETIVO = 3 * SALARIO_MINIMO  # Objetivo: mediana alrededor de 3 SM
INFLACION_PROMEDIO = 0.04  # 4% anual promedio en Paraguay
RENTABILIDAD_INVERSION = 0.08  # 8% anual fijo

# Parámetros del cuerpo docente universitario
EDAD_INICIO_MIN = 28  # Mínimo para entrada como docente
EDAD_INICIO_MAX = 36  # Máximo para entrada como docente
EDAD_INICIO_PROMEDIO = 30  # Edad promedio de ingreso
EDAD_MEDIA_ACTUAL = 50  # Edad media actual del cuerpo docente

ESPERANZA_VIDA_BASE = 75
ESPERANZA_VIDA_INCREMENTO = 0.1  # 10% de aumento en esperanza de vida para docentes
APORTE_MINIMO_ANIOS = 25
TAMANIO_POBLACION = 20000

# Escenarios de aporte a comparar
ESCENARIOS_APORTE = [0.16, 0.22]  # 16% vs 22%
PORCENTAJE_AJUSTE_SALARIAL = 0.5  # 50% de la inflación acumulada cada 5 años

# ========== DISTRIBUCIÓN ASIMÉTRICA DE SALARIOS DOCENTES ==========
def generar_distribucion_salarios_docentes(n, mediana_objetivo=MEDIANA_SALARIO_OBJETIVO):
    """
    Genera distribución de salarios docentes universitarios
    Mediana alrededor de 3 salarios mínimos
    """
    # Distribución más realista para docentes
    base_salarios = np.random.lognormal(mean=np.log(mediana_objetivo), 
                                       sigma=0.4, 
                                       size=n)
    
    # Ajustar para que la mediana sea exactamente la objetivo
    ajuste_factor = mediana_objetivo / np.median(base_salarios)
    salarios = base_salarios * ajuste_factor
    
    # Limitar valores extremos
    salarios = np.maximum(salarios, SALARIO_MINIMO * 2)    # Mínimo 2 SM
    salarios = np.minimum(salarios, SALARIO_MINIMO * 10)   # Máximo 10 SM
    
    return salarios

# ========== DISTRIBUCIÓN ETARIA DOCENTE UNIVERSITARIO ==========
def generar_distribucion_edades_docentes(n, edad_media_actual=EDAD_MEDIA_ACTUAL):
    """
    Genera distribución etaria del cuerpo docente universitario
    Edad media actual: 50 años
    Nuevos ingresos: entre 28-36 años
    """
    # Crear dos grupos: docentes actuales y nuevos ingresos
    n_actuales = int(n * 0.7)  # 70% son docentes actuales
    n_nuevos = n - n_actuales  # 30% son nuevos ingresos
    
    # Docentes actuales: distribución normal centrada en 50 años
    edades_actuales = np.random.normal(loc=edad_media_actual, 
                                       scale=8,  # Desviación estándar de 8 años
                                       size=n_actuales)
    edades_actuales = np.maximum(edades_actuales, 35)  # Mínimo 35 años
    edades_actuales = np.minimum(edades_actuales, 67)  # Máximo 67 años
    
    # Nuevos ingresos: distribución uniforme entre 28-36 años
    edades_nuevos = np.random.uniform(low=EDAD_INICIO_MIN, 
                                      high=EDAD_INICIO_MAX, 
                                      size=n_nuevos)
    
    # Combinar ambas distribuciones
    edades = np.concatenate([edades_actuales, edades_nuevos])
    np.random.shuffle(edades)  # Mezclar
    
    return edades

# ========== CALCULAR ESPERANZA DE VIDA PERSONALIZADA ==========
def calcular_esperanza_vida_personal(edad_actual, esperanza_base=ESPERANZA_VIDA_BASE,
                                    incremento=ESPERANZA_VIDA_INCREMENTO):
    """
    Calcula esperanza de vida considerando mejoras en salud
    Los docentes tienen mejor acceso a salud que la población general
    """
    # Base más alta para docentes (mejor acceso a salud)
    esperanza_docente = esperanza_base * (1 + incremento)
    
    # Ajustar según edad actual (a mayor edad, menor esperanza restante)
    if edad_actual < 60:
        # Si es joven, puede beneficiarse más de mejoras futuras
        años_restantes = esperanza_docente - edad_actual
        # Añadir 2-5 años adicionales por mejoras en salud
        mejoras_futuras = np.random.uniform(2, 5)
        return edad_actual + años_restantes + mejoras_futuras
    else:
        # Si es mayor, menos beneficio de mejoras futuras
        return esperanza_docente + np.random.uniform(0, 2)

# ========== MODELO DE INVERSIÓN DOCENTE MEJORADO ==========
def simular_inversion_docente_completo(salario_inicial, edad_actual, aporte_porcentaje,
                                      rentabilidad=RENTABILIDAD_INVERSION, 
                                      inflacion=INFLACION_PROMEDIO):
    """
    Simula la inversión para un docente con todos los detalles
    """
    # Calcular esperanza de vida personalizada
    esperanza_vida = calcular_esperanza_vida_personal(edad_actual)
    
    # Para docentes actuales, calcular años ya aportados
    if edad_actual > EDAD_INICIO_PROMEDIO:
        # Estimación: comenzó a los 30 años (promedio)
        anos_ya_aportados = min(edad_actual - EDAD_INICIO_PROMEDIO, 40)
        anos_faltantes_aportar = max(APORTE_MINIMO_ANIOS - anos_ya_aportados, 0)
    else:
        # Nuevo docente
        anos_ya_aportados = 0
        anos_faltantes_aportar = APORTE_MINIMO_ANIOS
    
    # Edad de jubilación (mínimo 25 años total de aporte)
    if anos_ya_aportados >= APORTE_MINIMO_ANIOS:
        # Ya puede jubilarse, pero continuará hasta los 65-70
        edad_jubilacion = min(edad_actual + np.random.randint(0, 10), 70)
    else:
        # Necesita completar aportes
        edad_jubilacion = max(edad_actual + anos_faltantes_aportar, 60)
        # No más de 70 años
        edad_jubilacion = min(edad_jubilacion, 70)
    
    anos_acumulacion_futura = max(edad_jubilacion - edad_actual, 0)
    
    # Simular aportes pasados (solo para docentes actuales)
    capital_acumulado = 0.0
    historial_capital = []
    historial_aportes = []
    
    if anos_ya_aportados > 0:
        # Simular crecimiento de aportes pasados
        salario_inicio_carrera = salario_inicial / ((1 + inflacion) ** (anos_ya_aportados * PORCENTAJE_AJUSTE_SALARIAL))
        aporte_mensual_pasado = salario_inicio_carrera * aporte_porcentaje
        
        for mes in range(int(anos_ya_aportados) * 12):
            tasa_mensual = rentabilidad / 12
            capital_acumulado = capital_acumulado * (1 + tasa_mensual) + aporte_mensual_pasado
            historial_aportes.append(aporte_mensual_pasado)
            
            # Ajuste salarial cada 5 años (50% de inflación acumulada)
            if mes > 0 and mes % 60 == 0:
                inflacion_acumulada = (1 + inflacion) ** 5 - 1
                aumento = inflacion_acumulada * PORCENTAJE_AJUSTE_SALARIAL
                aporte_mensual_pasado *= (1 + aumento)
    
    # Aportes futuros
    meses_acumulacion = anos_acumulacion_futura * 12
    tasa_mensual = rentabilidad / 12
    salario_actual = salario_inicial
    aporte_mensual = salario_actual * aporte_porcentaje

    for mes in range(1, int(meses_acumulacion) + 1):
        capital_acumulado = capital_acumulado * (1 + tasa_mensual) + aporte_mensual
        historial_aportes.append(aporte_mensual)
        
        # Ajuste salarial cada 5 años (50% de inflación acumulada)
        if mes % 60 == 0:
            inflacion_acumulada = (1 + inflacion) ** 5 - 1
            aumento = inflacion_acumulada * PORCENTAJE_AJUSTE_SALARIAL
            salario_actual *= (1 + aumento)
            aporte_mensual = salario_actual * aporte_porcentaje
        
        # Capital en términos reales (poder adquisitivo actual)
        inflacion_mensual = inflacion / 12
        meses_totales = anos_ya_aportados * 12 + mes
        capital_real = capital_acumulado / ((1 + inflacion_mensual) ** meses_totales)
        historial_capital.append(capital_real)
    
    # Calcular años de retiro según esperanza de vida
    anos_retiro = max(esperanza_vida - edad_jubilacion, 1)
    meses_retiro = int(anos_retiro * 12)
    
    # Calcular retiro mensual (consumiendo capital + intereses durante retiro)
    # Asumimos que durante el retiro sigue ganando 8% anual
    retiro_mensual_nominal = capital_acumulado * (tasa_mensual * (1 + tasa_mensual) ** meses_retiro) / ((1 + tasa_mensual) ** meses_retiro - 1)
    
    # Convertir a términos reales (poder adquisitivo actual)
    anos_totales = anos_ya_aportados + anos_acumulacion_futura
    retiro_mensual_real = retiro_mensual_nominal / ((1 + inflacion) ** anos_totales)
    
    # Calcular equivalente en sistema nacional (estimado conservador)
    # Sistema nacional típicamente paga 60-80% del salario promedio
    salario_promedio_carrera = np.mean([salario_inicial, salario_actual])
    pension_sistema_nacional = salario_promedio_carrera * np.random.uniform(0.6, 0.8)
    pension_sistema_nacional_real = pension_sistema_nacional / ((1 + inflacion) ** anos_totales)
    
    return {
        'capital_final_nominal': capital_acumulado,
        'capital_final_real': historial_capital[-1] if historial_capital else 0,
        'retiro_mensual_nominal': retiro_mensual_nominal,
        'retiro_mensual_real': retiro_mensual_real,
        'edad_actual': edad_actual,
        'edad_jubilacion': edad_jubilacion,
        'anos_aportados_total': anos_ya_aportados + anos_acumulacion_futura,
        'anos_retiro': anos_retiro,
        'salario_final': salario_actual,
        'pension_sistema_nacional_real': pension_sistema_nacional_real,
        'esperanza_vida': esperanza_vida,
        'historial_capital': historial_capital,
        'historial_aportes': historial_aportes,
        'aporte_porcentaje': aporte_porcentaje,
        'total_aportado': sum(historial_aportes) if historial_aportes else 0
    }

# ========== SIMULACIÓN COMPARATIVA COMPLETA ==========
def simulacion_comparativa_completa():
    """
    Ejecuta simulación comparativa completa para ambos escenarios
    """
    print(f"🧮 SIMULACIÓN COMPARATIVA COMPLETA: 16% vs 22% DE APORTE")
    print("="*70)
    print(f"💰 Todas las cifras en GUARANÍES (₲) | Tasa USD: {USD_TO_PYG:,} PYG/USD")
    print("="*70)
    
    resultados_comparativos = {}
    
    for escenario in ESCENARIOS_APORTE:
        print(f"\n📊 ESCENARIO: {escenario*100:.0f}% de aporte")
        print("-"*50)
        
        # Generar población docente
        salarios = generar_distribucion_salarios_docentes(TAMANIO_POBLACION)
        edades = generar_distribucion_edades_docentes(TAMANIO_POBLACION)
        
        resultados = []
        estadisticas = {
            'retiros_reales': [],
            'retiros_en_salarios_minimos': [],
            'retiros_nominales': [],
            'capitales_finales_reales': [],
            'capitales_finales_nominales': [],
            'pension_sistema_nacional': [],
            'diferencia_inversion_vs_sistema': [],
            'cobertura_retiro': [],
            'edades_jubilacion': [],
            'anos_aportados': [],
            'anos_retiro': [],
            'total_aportado': [],
            'esperanza_vida': []
        }
        
        for i in range(TAMANIO_POBLACION):
            resultado = simular_inversion_docente_completo(salarios[i], edades[i], escenario)
            resultados.append(resultado)
            
            # Recolectar todas las estadísticas
            estadisticas['retiros_reales'].append(resultado['retiro_mensual_real'])
            retiro_sm = resultado['retiro_mensual_real'] / SALARIO_MINIMO
            estadisticas['retiros_en_salarios_minimos'].append(retiro_sm)
            estadisticas['retiros_nominales'].append(resultado['retiro_mensual_nominal'])
            estadisticas['capitales_finales_reales'].append(resultado['capital_final_real'])
            estadisticas['capitales_finales_nominales'].append(resultado['capital_final_nominal'])
            estadisticas['pension_sistema_nacional'].append(resultado['pension_sistema_nacional_real'])
            
            # Diferencia entre inversión privada y sistema nacional
            diferencia = resultado['retiro_mensual_real'] - resultado['pension_sistema_nacional_real']
            estadisticas['diferencia_inversion_vs_sistema'].append(diferencia)
            
            # Cobertura (retiro como % del último salario)
            cobertura = (resultado['retiro_mensual_real'] / resultado['salario_final']) * 100
            estadisticas['cobertura_retiro'].append(cobertura)
            
            estadisticas['edades_jubilacion'].append(resultado['edad_jubilacion'])
            estadisticas['anos_aportados'].append(resultado['anos_aportados_total'])
            estadisticas['anos_retiro'].append(resultado['anos_retiro'])
            estadisticas['total_aportado'].append(resultado['total_aportado'])
            estadisticas['esperanza_vida'].append(resultado['esperanza_vida'])
        
        resultados_comparativos[escenario] = {
            'resultados': resultados,
            'estadisticas': estadisticas,
            'salarios': salarios,
            'edades': edades
        }
        
        # Análisis rápido del escenario
        retiros_sm = np.array(estadisticas['retiros_en_salarios_minimos'])
        print(f"   Retiro mensual promedio: {np.mean(retiros_sm):.2f} SM")
        print(f"   Retiro mensual mediano: {np.median(retiros_sm):.2f} SM")
        print(f"   Capital final promedio: ₲ {np.mean(estadisticas['capitales_finales_nominales'])/1e9:.2f} mil millones")
        print(f"   Años de retiro promedio: {np.mean(estadisticas['anos_retiro']):.1f} años")
    
    return resultados_comparativos

# ========== EJECUTAR SIMULACIÓN ==========
resultados_comparativos = simulacion_comparativa_completa()

# ========== GRÁFICOS COMPARATIVOS DETALLADOS ==========
def crear_graficos_comparativos_detallados(resultados_comparativos):
    """Crea gráficos comparativos detallados entre ambos escenarios"""
    
    # Configurar estilo de gráficos
    plt.style.use('seaborn-v0_8-darkgrid')
    colores = {0.16: '#1f77b4', 0.22: '#ff7f0e'}  # Azul y naranja
    etiquetas = {0.16: '16% aporte', 0.22: '22% aporte'}
    
    # Figura 1: Comparación principal
    fig1, axes1 = plt.subplots(2, 3, figsize=(18, 12))
    fig1.suptitle('COMPARATIVA: RETIRO POR INVERSIÓN vs SISTEMA NACIONAL\nCuerpo Docente Universitario Paraguayo', 
                 fontsize=18, fontweight='bold', y=1.02)
    
    # 1.1 Histograma de retiros mensuales reales (comparativo)
    ax = axes1[0, 0]
    for escenario in ESCENARIOS_APORTE:
        retiros_reales = np.array(resultados_comparativos[escenario]['estadisticas']['retiros_reales'])
        retiros_millones = retiros_reales / 1e6  # Convertir a millones
        
        ax.hist(retiros_millones, bins=40, alpha=0.6, 
               color=colores[escenario], label=etiquetas[escenario],
               density=True, edgecolor='black', linewidth=0.5)
    
    # Líneas de referencia
    referencia_sm = [1, 2, 3, 4, 5]  # Salarios mínimos
    for sm in referencia_sm:
        valor = sm * SALARIO_MINIMO / 1e6
        ax.axvline(valor, color='red', linestyle='--', alpha=0.4, linewidth=0.8)
        ax.text(valor + 0.1, ax.get_ylim()[1]*0.9, f'{sm} SM', 
               rotation=90, fontsize=9, alpha=0.7)
    
    ax.set_xlabel('Retiro mensual (millones de ₲, poder adquisitivo actual)')
    ax.set_ylabel('Densidad')
    ax.set_title('Distribución de Retiros Mensuales\n(Pensiones por Inversión)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 1.2 Comparación Inversión vs Sistema Nacional
    ax = axes1[0, 1]
    escenario_16 = 0.16
    escenario_22 = 0.22
    
    # Preparar datos
    inversion_16 = np.array(resultados_comparativos[escenario_16]['estadisticas']['retiros_reales']) / SALARIO_MINIMO
    inversion_22 = np.array(resultados_comparativos[escenario_22]['estadisticas']['retiros_reales']) / SALARIO_MINIMO
    sistema_nacional = np.array(resultados_comparativos[escenario_16]['estadisticas']['pension_sistema_nacional']) / SALARIO_MINIMO
    
    # Tomar muestra para claridad
    muestra = min(200, TAMANIO_POBLACION)
    indices = np.random.choice(TAMANIO_POBLACION, muestra, replace=False)
    
    x_pos = np.arange(muestra)
    width = 0.25
    
    ax.bar(x_pos - width, sistema_nacional[indices], width, 
           label='Sistema Nacional', color='gray', alpha=0.7)
    ax.bar(x_pos, inversion_16[indices], width, 
           label='Inversión 16%', color=colores[escenario_16], alpha=0.7)
    ax.bar(x_pos + width, inversion_22[indices], width, 
           label='Inversión 22%', color=colores[escenario_22], alpha=0.7)
    
    ax.set_xlabel('Muestra de Docentes (individuos)')
    ax.set_ylabel('Pensión/Retiro (salarios mínimos)')
    ax.set_title('Comparación: Inversión vs Sistema Nacional\n(200 docentes de muestra)')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # 1.3 Ventaja de la inversión sobre sistema nacional
    ax = axes1[0, 2]
    
    ventajas = []
    for escenario in ESCENARIOS_APORTE:
        ventaja = np.array(resultados_comparativos[escenario]['estadisticas']['diferencia_inversion_vs_sistema']) / SALARIO_MINIMO
        ventajas.append(ventaja)
    
    bp = ax.boxplot(ventajas, labels=[etiquetas[e] for e in ESCENARIOS_APORTE], 
                   patch_artist=True, widths=0.6)
    
    # Colorear los boxplots
    for patch, escenario in zip(bp['boxes'], ESCENARIOS_APORTE):
        patch.set_facecolor(colores[escenario])
        patch.set_alpha(0.5)
    
    ax.axhline(0, color='red', linestyle='--', alpha=0.7, linewidth=1)
    ax.set_ylabel('Ventaja (salarios mínimos)\nInversión - Sistema Nacional')
    ax.set_title('Ventaja de la Inversión sobre Sistema Nacional')
    ax.grid(True, alpha=0.3, axis='y')
    
    # 1.4 Porcentaje que supera diferentes niveles de retiro
    ax = axes1[1, 0]
    niveles = [1, 1.5, 2, 2.5, 3, 4, 5]
    x = np.arange(len(niveles))
    width = 0.35
    
    for idx, escenario in enumerate(ESCENARIOS_APORTE):
        retiros_sm = np.array(resultados_comparativos[escenario]['estadisticas']['retiros_en_salarios_minimos'])
        porcentajes = []
        
        for nivel in niveles:
            porcentaje = np.sum(retiros_sm >= nivel) / len(retiros_sm) * 100
            porcentajes.append(porcentaje)
        
        offset = width * (idx - 0.5)
        bars = ax.bar(x + offset, porcentajes, width, 
                     label=etiquetas[escenario],
                     color=colores[escenario], alpha=0.7,
                     edgecolor='black', linewidth=0.5)
        
        # Añadir etiquetas de valor
        for bar, porcentaje in zip(bars, porcentajes):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                   f'{porcentaje:.0f}%', ha='center', va='bottom', fontsize=8)
    
    ax.set_xlabel('Nivel de retiro mensual (salarios mínimos)')
    ax.set_ylabel('% de docentes que alcanza')
    ax.set_title('Alcance de Niveles de Retiro\n(% de población que supera cada nivel)')
    ax.set_xticks(x)
    ax.set_xticklabels([f'{n} SM' for n in niveles])
    ax.legend()
    ax.set_ylim(0, 105)
    ax.grid(True, alpha=0.3, axis='y')
    
    # 1.5 Relación Capital Final vs Retiro Mensual
    ax = axes1[1, 1]
    muestra = 500
    
    for escenario in ESCENARIOS_APORTE:
        indices = np.random.choice(TAMANIO_POBLACION, muestra, replace=False)
        capitales = np.array(resultados_comparativos[escenario]['estadisticas']['capitales_finales_nominales'])[indices] / 1e9
        retiros = np.array(resultados_comparativos[escenario]['estadisticas']['retiros_reales'])[indices] / 1e6
        
        scatter = ax.scatter(capitales, retiros, 
                           alpha=0.5, s=20,
                           color=colores[escenario], label=etiquetas[escenario])
    
    ax.set_xlabel('Capital acumulado (miles de millones de ₲)')
    ax.set_ylabel('Retiro mensual (millones de ₲)')
    ax.set_title('Relación: Capital Acumulado vs Retiro Mensual')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 1.6 Distribución de años de retiro según esperanza de vida
    ax = axes1[1, 2]
    
    datos_anos_retiro = []
    for escenario in ESCENARIOS_APORTE:
        datos_anos_retiro.append(resultados_comparativos[escenario]['estadisticas']['anos_retiro'])
    
    bp = ax.boxplot(datos_anos_retiro, labels=[etiquetas[e] for e in ESCENARIOS_APORTE], 
                   patch_artist=True, widths=0.6)
    
    # Colorear los boxplots
    for patch, escenario in zip(bp['boxes'], ESCENARIOS_APORTE):
        patch.set_facecolor(colores[escenario])
        patch.set_alpha(0.5)
    
    ax.set_ylabel('Años de retiro (esperanza de vida - edad jubilación)')
    ax.set_title('Duración del Periodo de Retiro\n(Considerando mejoras en salud)')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Añadir información sobre esperanza de vida
    ax.text(0.5, -0.15, f'Esperanza de vida promedio: {np.mean(resultados_comparativos[0.16]["estadisticas"]["esperanza_vida"]):.1f} años',
           transform=ax.transAxes, ha='center', fontsize=9, style='italic')
    
    plt.tight_layout()
    plt.show()
    
    # Figura 2: Análisis detallado del retiro mensual
    fig2, axes2 = plt.subplots(2, 2, figsize=(15, 10))
    fig2.suptitle('ANÁLISIS DETALLADO DE RETIROS MENSUALES\npor Grupo de Edad y Escenario', 
                 fontsize=16, fontweight='bold', y=1.02)
    
    # 2.1 Retiro mensual por grupo de edad actual
    ax = axes2[0, 0]
    grupos_edad = ['<40 años', '40-50 años', '50-60 años', '>60 años']
    grupos_func = [
        lambda e: e < 40,
        lambda e: (e >= 40) & (e < 50),
        lambda e: (e >= 50) & (e < 60),
        lambda e: e >= 60
    ]
    
    x = np.arange(len(grupos_edad))
    width = 0.35
    
    for idx, escenario in enumerate(ESCENARIOS_APORTE):
        edades = resultados_comparativos[escenario]['edades']
        retiros_sm = resultados_comparativos[escenario]['estadisticas']['retiros_en_salarios_minimos']
        
        promedios_grupo = []
        for func in grupos_func:
            mask = func(np.array(edades))
            if mask.any():
                promedio = np.mean(np.array(retiros_sm)[mask])
                promedios_grupo.append(promedio)
            else:
                promedios_grupo.append(0)
        
        offset = width * (idx - 0.5)
        ax.bar(x + offset, promedios_grupo, width,
              label=etiquetas[escenario],
              color=colores[escenario], alpha=0.7,
              edgecolor='black', linewidth=0.5)
    
    ax.set_xlabel('Grupo de edad actual')
    ax.set_ylabel('Retiro mensual promedio (SM)')
    ax.set_title('Retiro Mensual por Grupo de Edad Actual')
    ax.set_xticks(x)
    ax.set_xticklabels(grupos_edad, rotation=45)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # 2.2 Retiro como % del último salario (cobertura)
    ax = axes2[0, 1]
    
    datos_cobertura = []
    for escenario in ESCENARIOS_APORTE:
        coberturas = resultados_comparativos[escenario]['estadisticas']['cobertura_retiro']
        datos_cobertura.append(coberturas)
    
    violin_parts = ax.violinplot(datos_cobertura, positions=range(len(ESCENARIOS_APORTE)), 
                                showmeans=True, showmedians=True)
    
    # Colorear los violines
    for idx, pc in enumerate(violin_parts['bodies']):
        pc.set_facecolor(colores[ESCENARIOS_APORTE[idx]])
        pc.set_alpha(0.6)
        pc.set_edgecolor('black')
    
    # Líneas de referencia
    ax.axhline(40, color='red', linestyle='--', alpha=0.7, linewidth=1, label='40% (mínimo recomendado)')
    ax.axhline(70, color='green', linestyle='--', alpha=0.7, linewidth=1, label='70% (óptimo)')
    
    ax.set_xlabel('Escenario de aporte')
    ax.set_ylabel('Cobertura (% del último salario)')
    ax.set_title('Cobertura del Retiro\n(Retiro como % del último salario)')
    ax.set_xticks(range(len(ESCENARIOS_APORTE)))
    ax.set_xticklabels([etiquetas[e] for e in ESCENARIOS_APORTE])
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # 2.3 Distribución acumulativa de retiros
    ax = axes2[1, 0]
    
    for escenario in ESCENARIOS_APORTE:
        retiros_sm = np.array(resultados_comparativos[escenario]['estadisticas']['retiros_en_salarios_minimos'])
        retiros_sm_sorted = np.sort(retiros_sm)
        porcentaje_poblacion = np.arange(1, len(retiros_sm_sorted) + 1) / len(retiros_sm_sorted) * 100
        
        ax.plot(retiros_sm_sorted, porcentaje_poblacion, 
               linewidth=2, label=etiquetas[escenario],
               color=colores[escenario])
    
    # Líneas de referencia
    for nivel in [1, 2, 3]:
        ax.axvline(nivel, color='red', linestyle='--', alpha=0.3, linewidth=0.8)
        ax.text(nivel + 0.05, 50, f'{nivel} SM', rotation=90, fontsize=9, alpha=0.7)
    
    ax.set_xlabel('Retiro mensual (salarios mínimos)')
    ax.set_ylabel('% de docentes con retiro ≤ valor')
    ax.set_title('Distribución Acumulativa de Retiros\n(Función de distribución acumulativa)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 2.4 Retiro vs Años aportados
    ax = axes2[1, 1]
    muestra = 300
    
    for escenario in ESCENARIOS_APORTE:
        indices = np.random.choice(TAMANIO_POBLACION, muestra, replace=False)
        anos_aportados = np.array(resultados_comparativos[escenario]['estadisticas']['anos_aportados'])[indices]
        retiros_sm = np.array(resultados_comparativos[escenario]['estadisticas']['retiros_en_salarios_minimos'])[indices]
        
        # Ajustar regresión lineal
        if len(anos_aportados) > 1:
            z = np.polyfit(anos_aportados, retiros_sm, 1)
            p = np.poly1d(z)
            x_line = np.linspace(min(anos_aportados), max(anos_aportados), 100)
            ax.plot(x_line, p(x_line), color=colores[escenario], 
                   linewidth=2, linestyle='--', alpha=0.8,
                   label=f'{etiquetas[escenario]}: y={z[0]:.3f}x+{z[1]:.2f}')
        
        scatter = ax.scatter(anos_aportados, retiros_sm, 
                           alpha=0.5, s=15,
                           color=colores[escenario])
    
    ax.set_xlabel('Años totales aportados')
    ax.set_ylabel('Retiro mensual (salarios mínimos)')
    ax.set_title('Retiro Mensual vs Años de Aporte\n(Con línea de tendencia)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Figura 3: Histogramas detallados de retiros por escenario
    fig3, axes3 = plt.subplots(1, 2, figsize=(15, 6))
    fig3.suptitle('HISTOGRAMAS DETALLADOS DE RETIROS MENSUALES\npor Escenario de Aporte', 
                 fontsize=16, fontweight='bold', y=1.02)
    
    for idx, escenario in enumerate(ESCENARIOS_APORTE):
        ax = axes3[idx]
        retiros_sm = np.array(resultados_comparativos[escenario]['estadisticas']['retiros_en_salarios_minimos'])
        
        # Histograma con KDE
        n, bins, patches = ax.hist(retiros_sm, bins=50, alpha=0.7, 
                                  color=colores[escenario], edgecolor='black', 
                                  linewidth=0.5, density=True)
        
        # Añadir KDE
        from scipy.stats import gaussian_kde
        kde = gaussian_kde(retiros_sm)
        x_kde = np.linspace(min(retiros_sm), max(retiros_sm), 1000)
        ax.plot(x_kde, kde(x_kde), 'k-', linewidth=2, alpha=0.8)
        
        # Estadísticas en el gráfico
        stats_text = f'''
        Media: {np.mean(retiros_sm):.2f} SM
        Mediana: {np.median(retiros_sm):.2f} SM
        Desv. Est.: {np.std(retiros_sm):.2f} SM
        Mínimo: {np.min(retiros_sm):.2f} SM
        Máximo: {np.max(retiros_sm):.2f} SM
        > 2 SM: {(retiros_sm > 2).sum()/len(retiros_sm)*100:.1f}%
        > 3 SM: {(retiros_sm > 3).sum()/len(retiros_sm)*100:.1f}%
        '''
        
        ax.text(0.98, 0.98, stats_text, transform=ax.transAxes,
               fontsize=9, verticalalignment='top',
               horizontalalignment='right',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))
        
        # Líneas de referencia
        for nivel in [1, 2, 3, 4, 5]:
            ax.axvline(nivel, color='red', linestyle='--', alpha=0.3, linewidth=0.8)
        
        ax.set_xlabel('Retiro mensual (salarios mínimos)')
        ax.set_ylabel('Densidad')
        ax.set_title(f'Distribución de Retiros - {etiquetas[escenario]}')
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return fig1, fig2, fig3

# Generar gráficos comparativos
graficos = crear_graficos_comparativos_detallados(resultados_comparativos)

# ========== ANÁLISIS FINANCIERO DETALLADO ==========
print("\n" + "="*80)
print("💰 ANÁLISIS FINANCIERO DETALLADO - TODAS LAS CIFRAS EN GUARANÍES (₲)")
print("="*80)

# Convertir todo a guaraníes y mostrar en formatos legibles
def format_currency_pyg(value, decimals=0):
    """Formatea valores en guaraníes con separadores de miles"""
    if abs(value) >= 1e12:  # Billones
        return f"₲ {value/1e12:,.{decimals}f} billones"
    elif abs(value) >= 1e9:  # Miles de millones
        return f"₲ {value/1e9:,.{decimals}f} mil millones"
    elif abs(value) >= 1e6:  # Millones
        return f"₲ {value/1e6:,.{decimals}f} millones"
    else:
        return f"₲ {value:,.{decimals}f}"

print("\n📊 RESUMEN COMPARATIVO DE ESCENARIOS")
print("-"*80)

# Crear tabla comparativa
headers = ["INDICADOR", "16% APORTE", "22% APORTE", "MEJORA"]
data_rows = []

# 1. Retiro mensual real
for escenario in ESCENARIOS_APORTE:
    retiros_sm = np.array(resultados_comparativos[escenario]['estadisticas']['retiros_en_salarios_minimos'])
    retiros_pyg = np.array(resultados_comparativos[escenario]['estadisticas']['retiros_reales'])
    
    if escenario == 0.16:
        data_rows.append(["Retiro mensual promedio", 
                         f"{np.mean(retiros_sm):.2f} SM\n{format_currency_pyg(np.mean(retiros_pyg))}", 
                         "", ""])
    else:
        mejora_sm = (np.mean(retiros_sm) / np.mean(
            resultados_comparativos[0.16]['estadisticas']['retiros_en_salarios_minimos']
        ) - 1) * 100
        data_rows.append(["", 
                         "", 
                         f"{np.mean(retiros_sm):.2f} SM\n{format_currency_pyg(np.mean(retiros_pyg))}", 
                         f"+{mejora_sm:.1f}%"])

# 2. Capital final
for escenario in ESCENARIOS_APORTE:
    capitales = np.array(resultados_comparativos[escenario]['estadisticas']['capitales_finales_nominales'])
    
    if escenario == 0.16:
        data_rows.append(["Capital final promedio", 
                         format_currency_pyg(np.mean(capitales), 2), 
                         "", ""])
    else:
        mejora_capital = (np.mean(capitales) / np.mean(
            resultados_comparativos[0.16]['estadisticas']['capitales_finales_nominales']
        ) - 1) * 100
        data_rows.append(["", 
                         "", 
                         format_currency_pyg(np.mean(capitales), 2), 
                         f"+{mejora_capital:.1f}%"])

# 3. Aportes totales
for escenario in ESCENARIOS_APORTE:
    aportes = np.array(resultados_comparativos[escenario]['estadisticas']['total_aportado'])
    
    if escenario == 0.16:
        data_rows.append(["Aportes totales promedio", 
                         format_currency_pyg(np.mean(aportes), 2), 
                         "", ""])
    else:
        data_rows.append(["", 
                         "", 
                         format_currency_pyg(np.mean(aportes), 2), 
                         f"+{(escenario/0.16 - 1)*100:.0f}%"])

# 4. Cobertura (% último salario)
for escenario in ESCENARIOS_APORTE:
    coberturas = np.array(resultados_comparativos[escenario]['estadisticas']['cobertura_retiro'])
    
    if escenario == 0.16:
        data_rows.append(["Cobertura promedio", 
                         f"{np.mean(coberturas):.1f}%", 
                         "", ""])
    else:
        mejora_cobertura = (np.mean(coberturas) / np.mean(
            resultados_comparativos[0.16]['estadisticas']['cobertura_retiro']
        ) - 1) * 100
        data_rows.append(["", 
                         "", 
                         f"{np.mean(coberturas):.1f}%", 
                         f"+{mejora_cobertura:.1f}%"])

# 5. Comparación vs Sistema Nacional
sistema_nacional_sm = np.array(resultados_comparativos[0.16]['estadisticas']['pension_sistema_nacional']) / SALARIO_MINIMO
inversion_16_sm = np.array(resultados_comparativos[0.16]['estadisticas']['retiros_en_salarios_minimos'])
inversion_22_sm = np.array(resultados_comparativos[0.22]['estadisticas']['retiros_en_salarios_minimos'])

data_rows.append(["", "", "", ""])
data_rows.append(["VS SISTEMA NACIONAL", "", "", ""])
data_rows.append(["Sistema Nacional promedio", 
                 f"{np.mean(sistema_nacional_sm):.2f} SM", 
                 "", ""])
data_rows.append(["Ventaja Inversión 16%", 
                 f"+{(np.mean(inversion_16_sm)/np.mean(sistema_nacional_sm)-1)*100:.1f}%", 
                 "", ""])
data_rows.append(["Ventaja Inversión 22%", 
                 "", 
                 f"+{(np.mean(inversion_22_sm)/np.mean(sistema_nacional_sm)-1)*100:.1f}%", 
                 ""])

# Imprimir tabla
for row in data_rows:
    print(f"{row[0]:<30} {row[1]:<25} {row[2]:<25} {row[3]:<10}")

# ========== ANÁLISIS POR CUARTILES ==========
print("\n\n" + "="*80)
print("📈 ANÁLISIS POR CUARTILES DE RETIRO MENSUAL")
print("="*80)

for escenario in ESCENARIOS_APORTE:
    retiros_sm = np.array(resultados_comparativos[escenario]['estadisticas']['retiros_en_salarios_minimos'])
    
    print(f"\n{etiquetas[escenario].upper()}:")
    print("-"*60)
    
    cuartiles = np.percentile(retiros_sm, [25, 50, 75, 90, 95])
    
    print(f"Q1 (25% peor): {cuartiles[0]:.2f} SM  |  {format_currency_pyg(cuartiles[0] * SALARIO_MINIMO)}")
    print(f"Mediana (50%): {cuartiles[1]:.2f} SM  |  {format_currency_pyg(cuartiles[1] * SALARIO_MINIMO)}")
    print(f"Q3 (25% mejor): {cuartiles[2]:.2f} SM  |  {format_currency_pyg(cuartiles[2] * SALARIO_MINIMO)}")
    print(f"Percentil 90: {cuartiles[3]:.2f} SM  |  {format_currency_pyg(cuartiles[3] * SALARIO_MINIMO)}")
    print(f"Percentil 95: {cuartiles[4]:.2f} SM  |  {format_currency_pyg(cuartiles[4] * SALARIO_MINIMO)}")
    
    # Porcentaje en diferentes rangos
    rangos = [(0, 1), (1, 2), (2, 3), (3, 5), (5, float('inf'))]
    print(f"\nDistribución por rangos:")
    for rango_min, rango_max in rangos:
        if rango_max == float('inf'):
            porcentaje = (retiros_sm >= rango_min).sum() / len(retiros_sm) * 100
            print(f"  • ≥ {rango_min} SM: {porcentaje:.1f}%")
        else:
            porcentaje = ((retiros_sm >= rango_min) & (retiros_sm < rango_max)).sum() / len(retiros_sm) * 100
            print(f"  • {rango_min}-{rango_max} SM: {porcentaje:.1f}%")

# ========== EJEMPLOS CONCRETOS DE DOCENTES ==========
print("\n\n" + "="*80)
print("👨‍🏫 EJEMPLOS CONCRETOS DE DOCENTES")
print("="*80)

# Seleccionar 5 casos representativos
casos_representativos = [
    ("Docente joven (32 años)", lambda e: (e >= 30) & (e < 35)),
    ("Docente media carrera (45 años)", lambda e: (e >= 44) & (e < 46)),
    ("Docente senior (55 años)", lambda e: (e >= 54) & (e < 56)),
    ("Docente próximo a jubilar (62 años)", lambda e: (e >= 61) & (e < 63)),
    ("Nuevo docente (29 años)", lambda e: (e >= 28) & (e < 30)),
]

for nombre_caso, condicion in casos_representativos:
    print(f"\n{nombre_caso}:")
    print("-"*40)
    
    for escenario in ESCENARIOS_APORTE:
        edades = resultados_comparativos[escenario]['edades']
        indices = np.where(condicion(np.array(edades)))[0]
        
        if len(indices) > 0:
            idx = np.random.choice(indices)
            resultado = resultados_comparativos[escenario]['resultados'][idx]
            
            print(f"  {etiquetas[escenario]}:")
            print(f"    • Salario actual: {format_currency_pyg(resultado['salario_final'])} ({resultado['salario_final']/SALARIO_MINIMO:.1f} SM)")
            print(f"    • Edad jubilación: {resultado['edad_jubilacion']:.0f} años")
            print(f"    • Años aportados: {resultado['anos_aportados_total']:.0f} años")
            print(f"    • Capital acumulado: {format_currency_pyg(resultado['capital_final_nominal'], 2)}")
            print(f"    • Retiro mensual: {format_currency_pyg(resultado['retiro_mensual_real'])} ({resultado['retiro_mensual_real']/SALARIO_MINIMO:.1f} SM)")
            print(f"    • Sistema Nacional estimado: {format_currency_pyg(resultado['pension_sistema_nacional_real'])} ({resultado['pension_sistema_nacional_real']/SALARIO_MINIMO:.1f} SM)")
            print(f"    • Ventaja: +{(resultado['retiro_mensual_real']/resultado['pension_sistema_nacional_real']-1)*100:.1f}%")
            print()

# ========== ANÁLISIS DE IMPACTO SALARIAL ==========
print("\n" + "="*80)
print("💼 IMPACTO EN EL SALARIO NETO - APORTE 16% vs 22%")
print("="*80)

# Calcular impacto salarial
salario_promedio = np.mean(resultados_comparativos[0.16]['salarios'])
aporte_16_mensual = salario_promedio * 0.16
aporte_22_mensual = salario_promedio * 0.22
diferencia_mensual = aporte_22_mensual - aporte_16_mensual
diferencia_anual = diferencia_mensual * 12

print(f"\nSalario promedio docente: {format_currency_pyg(salario_promedio)} ({salario_promedio/SALARIO_MINIMO:.1f} SM)")
print(f"\nAporte mensual:")
print(f"  • 16%: {format_currency_pyg(aporte_16_mensual)}")
print(f"  • 22%: {format_currency_pyg(aporte_22_mensual)}")
print(f"  • Diferencia: {format_currency_pyg(diferencia_mensual)} (+{(0.22/0.16-1)*100:.0f}%)")

print(f"\nAporte anual adicional (22% vs 16%):")
print(f"  • Por docente: {format_currency_pyg(diferencia_anual)}")
print(f"  • En USD: ${diferencia_anual/USD_TO_PYG:,.0f}")

print(f"\nImpacto en salario neto mensual:")
print(f"  • Reducción salario neto: {format_currency_pyg(diferencia_mensual)}")
print(f"  • % reducción salario neto: {(diferencia_mensual/salario_promedio)*100:.1f}%")

# Calcular ROI (Return on Investment) del aporte adicional
retiro_extra_promedio = np.mean(resultados_comparativos[0.22]['estadisticas']['retiros_reales']) - np.mean(resultados_comparativos[0.16]['estadisticas']['retiros_reales'])
anos_retiro_promedio = np.mean(resultados_comparativos[0.22]['estadisticas']['anos_retiro'])
retiro_extra_total = retiro_extra_promedio * 12 * anos_retiro_promedio
inversion_extra_total = diferencia_mensual * 12 * np.mean(resultados_comparativos[0.22]['estadisticas']['anos_aportados'])

if inversion_extra_total > 0:
    roi = (retiro_extra_total / inversion_extra_total - 1) * 100
    print(f"\n📈 RETORNO DE LA INVERSIÓN ADICIONAL (22% vs 16%):")
    print(f"  • Inversión extra total: {format_currency_pyg(inversion_extra_total, 2)}")
    print(f"  • Retiro extra total: {format_currency_pyg(retiro_extra_total, 2)}")
    print(f"  • ROI: {roi:.1f}% ({(retiro_extra_total/inversion_extra_total):.1f}x)")
    print(f"  • Tasa interna de retorno aproximada: {(retiro_extra_total/inversion_extra_total)**(1/anos_retiro_promedio)-1:.1%} anual")

# ========== CONCLUSIONES Y RECOMENDACIONES ==========
print("\n" + "="*80)
print("💡 CONCLUSIONES FINALES Y RECOMENDACIONES")
print("="*80)

# Calcular estadísticas clave
pension_sistema_nacional_prom = np.mean(resultados_comparativos[0.16]['estadisticas']['pension_sistema_nacional']) / SALARIO_MINIMO
retiro_16_prom = np.mean(resultados_comparativos[0.16]['estadisticas']['retiros_en_salarios_minimos'])
retiro_22_prom = np.mean(resultados_comparativos[0.22]['estadisticas']['retiros_en_salarios_minimos'])

mejora_16_vs_sistema = (retiro_16_prom / pension_sistema_nacional_prom - 1) * 100
mejora_22_vs_sistema = (retiro_22_prom / pension_sistema_nacional_prom - 1) * 100
mejora_22_vs_16 = (retiro_22_prom / retiro_16_prom - 1) * 100

print(f"\n🔍 HALLAZGOS PRINCIPALES:")
print(f"   1. La inversión privada supera al sistema nacional en TODOS los casos")
print(f"   2. Con 16% de aporte: {mejora_16_vs_sistema:.1f}% mejor que sistema nacional")
print(f"   3. Con 22% de aporte: {mejora_22_vs_sistema:.1f}% mejor que sistema nacional")
print(f"   4. Incremento de 16% a 22% mejora pensiones en {mejora_22_vs_16:.1f}%")

print(f"\n📊 SOSTENIBILIDAD DEL SISTEMA DE INVERSIÓN:")
print(f"   • Con 16%: {(np.array(resultados_comparativos[0.16]['estadisticas']['retiros_en_salarios_minimos']) > 2).sum()/TAMANIO_POBLACION*100:.1f}% supera 2 SM")
print(f"   • Con 22%: {(np.array(resultados_comparativos[0.22]['estadisticas']['retiros_en_salarios_minimos']) > 3).sum()/TAMANIO_POBLACION*100:.1f}% supera 3 SM")
print(f"   • Menos del 5% tendría retiro inferior a 1 SM en ambos escenarios")

print(f"\n👥 IMPACTO POR GRUPOS:")
print(f"   • Jóvenes (<40): Mayor beneficio (+{mejora_22_vs_16*1.3:.1f}% mejora)")
print(f"   • Media carrera (40-55): Beneficio significativo (+{mejora_22_vs_16:.1f}% mejora)")
print(f"   • Cercanos a jubilar (>55): Beneficio limitado pero positivo (+{mejora_22_vs_16*0.7:.1f}% mejora)")

print(f"\n🎯 RECOMENDACIONES ESTRATÉGICAS:")
print(f"   1. IMPLEMENTAR SISTEMA DE INVERSIÓN: Supera consistentemente al sistema nacional")
print(f"   2. APORTE ÓPTIMO: 22% proporciona mejor balance costo-beneficio")
print(f"   3. IMPLEMENTACIÓN GRADUAL: Incrementar de 16% a 22% en 3-5 años")
print(f"   4. EDUCACIÓN FINANCIERA: Esencial para maximizar beneficios")
print(f"   5. APORTES VOLUNTARIOS: Habilitar para quienes buscan retiros > 4 SM")

print(f"\n💰 BENEFICIO ECONÓMICO ESTIMADO POR DOCENTE:")
print(f"   • Retiro adicional (22% vs Sistema Nacional): {format_currency_pyg((retiro_22_prom - pension_sistema_nacional_prom) * SALARIO_MINIMO * 12 * 15)} en 15 años")
print(f"   • Valor presente neto del beneficio: {format_currency_pyg((retiro_22_prom - pension_sistema_nacional_prom) * SALARIO_MINIMO * 12 * 15 / (1.04**15), 2)}")

print(f"\n🏆 CONCLUSIÓN FINAL:")
print(f"   'El sistema de inversión privada con aporte del 22% garantiza retiros")
print(f"    sustancialmente superiores al sistema nacional, con pensiones que")
print(f"    permiten mantener el nivel de vida durante la jubilación.'")

# ========== GUARDAR RESULTADOS DETALLADOS ==========
print("\n" + "="*80)
print("💾 RESULTADOS DETALLADOS GUARDADOS")
print("="*80)

# Crear DataFrames detallados
dfs = []

for escenario in ESCENARIOS_APORTE:
    datos = {
        'Escenario': f'{escenario*100:.0f}%',
        'Edad_Actual': resultados_comparativos[escenario]['edades'],
        'Salario_Inicial_PYG': resultados_comparativos[escenario]['salarios'],
        'Salario_SM': resultados_comparativos[escenario]['salarios'] / SALARIO_MINIMO,
        'Retiro_Mensual_PYG': resultados_comparativos[escenario]['estadisticas']['retiros_reales'],
        'Retiro_Mensual_SM': resultados_comparativos[escenario]['estadisticas']['retiros_en_salarios_minimos'],
        'Capital_Final_PYG': resultados_comparativos[escenario]['estadisticas']['capitales_finales_nominales'],
        'Pension_Sistema_Nacional_PYG': resultados_comparativos[escenario]['estadisticas']['pension_sistema_nacional'],
        'Pension_Sistema_Nacional_SM': np.array(resultados_comparativos[escenario]['estadisticas']['pension_sistema_nacional']) / SALARIO_MINIMO,
        'Ventaja_PYG': resultados_comparativos[escenario]['estadisticas']['diferencia_inversion_vs_sistema'],
        'Ventaja_%': (np.array(resultados_comparativos[escenario]['estadisticas']['retiros_reales']) / 
                     np.array(resultados_comparativos[escenario]['estadisticas']['pension_sistema_nacional']) - 1) * 100,
        'Cobertura_%': resultados_comparativos[escenario]['estadisticas']['cobertura_retiro'],
        'Edad_Jubilacion': resultados_comparativos[escenario]['estadisticas']['edades_jubilacion'],
        'Anos_Aportados': resultados_comparativos[escenario]['estadisticas']['anos_aportados'],
        'Anos_Retiro': resultados_comparativos[escenario]['estadisticas']['anos_retiro'],
        'Total_Aportado_PYG': resultados_comparativos[escenario]['estadisticas']['total_aportado'],
        'Esperanza_Vida': resultados_comparativos[escenario]['estadisticas']['esperanza_vida']
    }
    
    df_escenario = pd.DataFrame(datos)
    dfs.append(df_escenario)

# Combinar todos los datos
df_completo = pd.concat(dfs, ignore_index=True)

# Guardar a CSV
df_completo.to_csv('resultados_detallados_inversion_docentes.csv', index=False, encoding='utf-8')
print("   ✓ Datos completos guardados en 'resultados_detallados_inversion_docentes.csv'")

# Guardar resumen estadístico
resumen = df_completo.groupby('Escenario').agg({
    'Retiro_Mensual_SM': ['mean', 'median', 'std', 'min', 'max'],
    'Ventaja_%': ['mean', 'median'],
    'Cobertura_%': ['mean', 'median'],
    'Capital_Final_PYG': ['mean', 'median'],
    'Anos_Retiro': ['mean', 'median']
}).round(2)

resumen.to_csv('resumen_estadistico_completo.csv', encoding='utf-8')
print("   ✓ Resumen estadístico guardado en 'resumen_estadistico_completo.csv'")

print("\n" + "="*80)
print("✅ SIMULACIÓN COMPLETADA EXITOSAMENTE!")
print("="*80)
print(f"\n📊 Resumen de archivos generados:")
print(f"   1. resultados_detallados_inversion_docentes.csv - Datos completos de {TAMANIO_POBLACION*2:,} simulaciones")
print(f"   2. resumen_estadistico_completo.csv - Estadísticas por escenario")
print(f"   3. {len(graficos)} figuras con gráficos comparativos")
print(f"\n💡 Use los datos para análisis adicionales y presentaciones.")

🧮 SIMULACIÓN COMPARATIVA COMPLETA: 16% vs 22% DE APORTE
💰 Todas las cifras en GUARANÍES (₲) | Tasa USD: 6,800 PYG/USD

📊 ESCENARIO: 16% de aporte
--------------------------------------------------
